# Классификация текстов при помощи алгоритма KNN

## Постановка задачи
Необходимо написать программу для классификации текстов. Пайплан этого процесса можно определить следующим образом:
<ul>
<li>извлечение текста из файлов </li>
<li>удаление нетекстовых символов</li>
<li>удаление "стоп-слов"</li>
<li>разбиение текста на токены</li>
<li>лемматизация</li>
<li>вычисление TF-IDF матрицы</li>
<li>подготовка модели KNN</li>
<li>определение жанра неизвестного произведения</li>
</ul>

## Алгоритм KNN

Метод k-ближайших соседей (англ. k-nearest neighbors algorithm, k-NN) — метрический алгоритм для автоматической классификации объектов или регрессии.

В случае использования метода для классификации объект присваивается тому классу, который является наиболее распространённым среди k  соседей данного элемента, классы которых уже известны.

## Структура корпуса
Текстовой корпус включает в себя произведения двух жанров: научной фантастики и детектива.
Для каждого жанра были отобраны пять наиболее известных авторов (или скорее, на мой вкус). <br>
Каждый автор представлен двумя небольшими произведениями (рассказами или повестями). <br>
Объем каждого произведения составляет около 10-20 страниц. Формат текстовых книг: epub.

Для жанра научной фантастики были отобраны следующие писатели:
<ul>
<li>Айзек Азимов</li>
<li>Рей Бредберри </li>
<li>Харлон Элиссон</li>
<li>Станислав Лем</li>
<li>Клиффорд Саймак</li>
</ul>
<img src="images/Science_fiction_corpora.jpg" width=772 height=324/> <br>

Для жанра детективных историй:
<ul>
<li>Конан Дойль</li>
<li>Гиберт Кей Честертон</li>
<li>Эдгар Аллан По</li>
<li>Рекс Стаут</li>
<li>Джон Карр</li>
</ul>
<img src="images/Detective_stories_corpora.jpg" width=772 height=324/> <br>


## Программная реализация

### Импорт модулей и библиотек

In [61]:
import re
import os
import requests
import pymorphy2
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from epub_conversion.utils import open_book, convert_epub_to_lines

### Морфологический анализатор
Лемматизация -- это процесс приведения словоформы к лемме, то есть её нормальной (словарной) форме.<br>
Лемматизация анализируемого текста позволяет снизить степень разряженности желаемой матрицы признаков.

Существует несколько модулей языка Python, способных лемматизировать текст на русском языке.
В данной работе используется модуль *pymorphy2*.

Стоит отметить, что данный этап - довольно ресурсоемкий с вычислительной точки зрения,
так как поиск по словарям занимает существенное время.


In [62]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

### Получение списка стоп-слов для русского языка

Стоп-слова – это слова, которые исключаются из текста до или после обработки текста. Такие слова могут добавить много шума, поэтому необходимо избавляться от нерелевантных слов.

Под стоп-словами это обычно понимают артикли, междометия, союзы и т.д., которые не несут смысловой нагрузки.
При этом надо понимать, что не существует универсального списка стоп-слов, все зависит от конкретного случая.

In [63]:
def get_text_from_url(url, encoding='utf-8', to_lower=True):
    url = str(url)
    if url.startswith('http'):
        r = requests.get(url)
        if not r.ok:
            r.raise_for_status()
        return r.text.lower() if to_lower else r.text
    elif os.path.exists(url):
        with open(url, encoding=encoding) as f:
            return f.read().lower() if to_lower else f.read()
    else:
        raise Exception('parameter [url] can be either URL or a filename')

rus_stopwords_url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"
rus_stopwords = get_text_from_url(rus_stopwords_url).splitlines()

### Получение текста из файлов электронных книг

Функция читает файл epub, извлекает оттуда текст и удаляет из него xml теги при помощи регулярного выражения. <br>
Полученный и очищенный текст созраняется в виде .txt файла.

In [64]:
def get_text_from_epub(epub_file_path: Path, output_dir_path='./') -> str:
    book = open_book(str(epub_file_path))
    lines = convert_epub_to_lines(book)
    cleaner = re.compile('<.*?>')

    output_txt_filename = f'{output_dir_path}/{epub_file_path.stem}.txt'
    with open(output_txt_filename, 'w', encoding='utf-8') as txt_file:
        for line in lines:
            line = re.sub(cleaner, '', line)
            txt_file.write(line)

    return output_txt_filename

### Предварительная очистка и нормализация текстовых строк

Из полученных текстовых строк необходимо удалить символы пунктуации, символы цифр и тп. <br>

В текстах книг также могут встречаться странные аномалии, когда слово может содержать символы цифр, играющих роль букв
(например, "3акон"). Такие аномалии необходимо выявлять и исключать. <br>

Слова на английском языке также должны быть исключены из текста. Такие слова можно выявить путем проверки строки на наличие
ascii символов.

In [65]:
def remove_punctuation(text_string: str) -> str:
    return re.sub(r'[^\w\s]', ' ', text_string)

def has_numbers(text_string: str) -> bool:
    return any(char.isdigit() for char in text_string)

def has_ascii(text_string: str) -> bool:
    return any(char.isascii() for char in text_string)

def lemmatization(txt_file_path: str) -> str:
    with open(txt_file_path, 'r', encoding='utf-8') as txt_file:
        token_string = str()
        for line in txt_file.readlines():
            line = remove_punctuation(line)
            for word in line.split():
                if not any([word.isnumeric(), word.isascii(), has_numbers(word), has_ascii(word)]):
                    token_string += ' ' + morph.parse(word.lower())[0].normal_form
    return token_string

In [66]:
book_names = list()

In [67]:
def list_directory(source_dir_path_name: str, target_dir_path_name: str, dataset: list) -> list:
    path = Path(source_dir_path_name)
    for ebook in path.iterdir():
        txt_file_name = get_text_from_epub(ebook, target_dir_path_name)
        tokens = lemmatization(txt_file_name)
        dataset.append(tokens)
        book_names.append(ebook.stem)
        print(f'{ebook.stem} was analysed.')
    return dataset

dataset = list()
print('*** SCIENCE FICTION ***')
dataset = list_directory('./texts/science_fiction/russian/epub', './texts/science_fiction/russian/txt', dataset)
print('*** ***' * 4)

print('*** DETECTIVE STORIES ***')
dataset = list_directory('./texts/detective_stories/russian/epub', './texts/detective_stories/russian/txt',dataset)
print('*** ***' * 4)

*** SCIENCE FICTION ***
Azimov_Robbie was analysed.
Azimov_Runaround was analysed.
Bradbury_A_Sound_of_Thunder was analysed.
Bradbury_The_Concrete_Mixer was analysed.
Ellison_A_boy_and_his_dog was analysed.
Ellison_I_Have_No_Mouth was analysed.
Lem_End_of_the_World was analysed.
Lem_Mask was analysed.
Simak_Grotto_of_the_Dancing_Deer was analysed.
Simak_New_Folks_Home was analysed.
*** ****** ****** ****** ***
*** DETECTIVE STORIES ***
Carr_Silver_curtain was analysed.
Carr_The_Mystery_of_Great_Virley was analysed.
Chesterton_Cross was analysed.
Chesterton_Heaven was analysed.
Conan_Doyle_Orange was analysed.
Conan_Doyle_Scandal was analysed.
Poe_Golden_Bug was analysed.
Poe_Murders_in_the_Morgue was analysed.
Rex_Stout_American_style was analysed.
Rex_Stout_Love_story was analysed.
*** ****** ****** ****** ***


### Получение матрицы признаков TF-IDF

TF-IDF (term frequency, inverse document) -- это статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса.
Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.

In [68]:
vectorizer = TfidfVectorizer(stop_words=rus_stopwords)
x_vector = vectorizer.fit_transform(dataset)
arr = x_vector.toarray()

y_train = np.zeros(20)
y_train[10:] = 1

print(f"Число уникальных слов: {arr.shape[1]}")
print(vectorizer.get_feature_names_out())
print(x_vector.toarray())

Число уникальных слов: 13790
['аарон' 'абзац' 'абонент' ... 'ящик' 'ящичек' 'ёкнуть']
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.00337788 0.         0.        ]
 [0.         0.         0.         ... 0.00588033 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.0162971  0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.03298587 0.         0.        ]]


C:\Users\Максим\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  warnings.warn(


В результате всех предварительных действий имеется матрица признаков большей размерности, где строка соответствует одному произведению, а столбец -- одному уникальному слову.

### Алгоритм KNN из библиотеки sklearn
Сначала воспользуемся реализацией алгоритма KNN из библиотеки sklearn.

Основным варьируемым параметром для данного алгоритма является число ближайших соседей K.

In [69]:
k = 3
model_knn = KNeighborsClassifier(n_neighbors=k)
model_knn.fit(x_vector,y_train)

KNeighborsClassifier(n_neighbors=3)

Тестирование работы алгоритма будет проводиться путем проверки принадлежности к жанру нескольких рассказов Артура Кларка (фантастика) и Агаты Кристи (детектив), которые до этого не рассматривались и не были включены в корпус.


In [70]:
genres = {
    0: 'научная фантастика',
    1: 'детектив'
}

print(" *** TESTING *** ")
test_dataset = list_directory('./test/epub', './test/txt', list())
test_vector = vectorizer.transform(test_dataset)
prediction = model_knn.predict(test_vector)
prediction_probabilities = model_knn.predict_proba(test_vector)
print(" *** *** *** " * 4)
for number, ebook in enumerate(Path('./test/epub').iterdir()):
    pred_index = int(prediction[number])
    print(f"{ebook.stem} относится к жанру {genres[pred_index]}"
       f" c вероятностью {round(prediction_probabilities[number][pred_index], 2)}")

 *** TESTING *** 
Christie_The_Double_Clue was analysed.
Christie_The_Veiled_Lady was analysed.
Clarke_Constellation_of_the_dog was analysed.
Clarke_The_Nine_Billion_Names_of_God was analysed.
 *** *** ***  *** *** ***  *** *** ***  *** *** *** 
Christie_The_Double_Clue относится к жанру детектив c вероятностью 1.0
Christie_The_Veiled_Lady относится к жанру детектив c вероятностью 1.0
Clarke_Constellation_of_the_dog относится к жанру научная фантастика c вероятностью 0.67
Clarke_The_Nine_Billion_Names_of_God относится к жанру детектив c вероятностью 0.67


### Самодельный алгоритм KNN

Ради интереса я решил написать свои функции вычисления евклидового расстояния между векторам для того, чтобы определить, на какие именно произведения похоже исследуемое в данный момент.

In [71]:
def euclidean_distance(first_vector: np.ndarray, second_vector: np.ndarray) -> np.ndarray:
    return np.linalg.norm(first_vector - second_vector)

def calculate_distances(matrix: np.ndarray, vector: np.ndarray) -> dict:
    distances = dict()
    for num, row in enumerate(matrix):
        distances[num] = euclidean_distance(row, vector)
    return distances

In [72]:
similarity = calculate_distances(arr, test_vector[0])
similarity = {k: v for k, v in sorted(similarity.items(), key=lambda item: item[1])}
nearest_books_indices = list(similarity.keys())[:k]

In [73]:
genre_probability = sum([1 for i in nearest_books_indices if i >= 10]) / len(nearest_books_indices) * 100
if genre_probability >= 50.0:
    print(f'При k={k} ближайших соседях выбранная книга относится к жанру детектив с вероятностью {genre_probability} %')
    print(f'Cамые похожие на нее книги: {[book_names[kn] for kn in nearest_books_indices]}')
else:
    print(f'При k={k} ближайших соседях выбранная книга относится к жанру научной фантастики с вероятностью {genre_probability} %')
    print(f'Cамые похожие на нее кнгиги: {[book_names[kn] for kn in nearest_books_indices]}')

При k=3 ближайших соседях выбранная книга относится к жанру детектив с вероятностью 100.0 %
Cамые похожие на нее книги: ['Carr_Silver_curtain', 'Rex_Stout_Love_story', 'Conan_Doyle_Scandal']


## Выводы

Несмотря на свою простоту, алгоритм KNN действительно способен определять жанр произведения по известному корпусу текстов, как это случилось с детективными историями.

Однако, как можно было заметить, данный алгоритм страдает от размытости границ жанра: те произведения, которые я отнес к фантастике, очень сильно отличаются друг от друга и строго говоря относятся к разным её поджанрам.
Кроме того, как показали эксперименты, с увеличением числа k точность определения трагически падает и перестает выдавать адекватный результат. Это можно объяснить тем, что полученные точки в многомерном пространстве находится достаточно близко друг к другу и степень различия между текстами не так велика, как может показаться.

## Что можно сделать дальше


* переписать код согласно подходу к созданию pipeline
* создать функции визуализации результатов проверки принадлежности к жанру
*  проверить, как будет меняться результат определения жанра если:
    * изменять число k
    * не прибегать к лемматизации (использовать стемминг или вообще ничего)
    * использовать корпус тех произведения, но на английском языке
    * воспользоваться другим способом вычисления матрицы признаков

